In [4]:
import pandas as pd
from sklearn.metrics import make_scorer, f1_score
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import RandomizedSearchCV
import string
import emoji
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
vectorizer = TfidfVectorizer(stop_words=russian_stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def lemmatize_text(text):
    return ' '.join([morph.parse(word)[0].normal_form for word in text.split()])
punctuation = string.punctuation
test_df = pd.read_csv('../data/test.csv')
train_df = pd.read_csv('../data/train.csv')
train_df['text'] = train_df['text'].apply(lambda x: x.lower())
train_df['text'] = train_df['text'].apply(lambda x: x.translate(str.maketrans('', '', punctuation)))
train_df['text'] = train_df['text'].apply(lambda x: emoji.replace_emoji(x, replace=''))
train_df['text'] = train_df['text'].apply(lemmatize_text)
test_df['text'] = test_df['text'].apply(lambda x: x.lower())
test_df['text'] = test_df['text'].apply(lambda x: x.translate(str.maketrans('', '', punctuation)))
test_df['text'] = test_df['text'].apply(lambda x: emoji.replace_emoji(x, replace=''))
test_df['text'] = test_df['text'].apply(lemmatize_text)
X_train = train_df['text']
X_test = test_df['text']
Y_train = train_df['label']
Y_test = test_df['label']
vectorizer = TfidfVectorizer(analyser='char', ngram_range=(4, 6), stop_words=russian_stopwords)

In [114]:
print(X_train)
vectorizer.fit(X_train)
X_train_vec = vectorizer.transform(X_train)
X_test_vec = vectorizer.transform(X_test)
print(X_train)
# model = XGBClassifier(objective='binary:logistic', eval_metric='logloss', use_label_encoder=False)
# param_dist = {
#     'n_estimators': [100, 200, 500],
#     'max_depth': [3, 5, 7, 10],
#     'learning_rate': [0.01, 0.05, 0.1, 0.3],
#     'subsample': [0.7, 0.8, 0.9, 1.0],
#     'colsample_bytree': [0.7, 0.8, 0.9, 1.0],
#     'gamma': [0, 0.1, 0.3],
#     'reg_alpha': [0, 0.1, 0.5],
#     'reg_lambda': [0, 0.1, 0.5]
# }
# scorer = make_scorer(f1_score)
# random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=100, scoring=scorer, cv=3, verbose=2, random_state=42, n_jobs=-1)
# random_search.fit(X_train_vec, Y_train)
# print(random_search.best_params_)
params = {'subsample': 0.8, 'reg_lambda': 0.1, 'reg_alpha': 0.1, 'n_estimators': 500, 'max_depth': 10, 'learning_rate': 0.1, 'gamma': 0.1, 'colsample_bytree': 0.7}
model = XGBClassifier(objective='binary:logistic', eval_metric='logloss', use_label_encoder=False, **params)


model.fit(X_train_vec, Y_train)
pred = model.predict(X_test_vec)
print(f1_score(pred, Y_test))

0         видимо в разный регион называть по разный в ро...
1         понятно что это нарушение правило писать капсл...
2                             какой классный жизненный стих
3                              а и правдакогда он запретить
4         в солёный вода вирус жить учёный изучать соста...
                                ...                        
223456    вова дима когда же вы подавиться деньга челове...
223457      какой красота просто нет слово выразить чувство
223458    вы пост гаи выставитя на перекрёсток возле 21 ...
223459                           как то на лебедь непохожий
223460                            интересно чей это самолёт
Name: text, Length: 223461, dtype: object
0         видимо в разный регион называть по разный в ро...
1         понятно что это нарушение правило писать капсл...
2                             какой классный жизненный стих
3                              а и правдакогда он запретить
4         в солёный вода вирус жить учёный изучать соста..

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [02:25:07] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


0.845704339176924


In [5]:
huy = pd.array(['руки бы оторвать'])
huy = vectorizer.transform(huy)
print(model.predict(huy))

NotFittedError: The TF-IDF vectorizer is not fitted